In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import os 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
print(os.listdir('../input'))

In [ ]:
! unzip '../input/dogs-vs-cats/train.zip' 
! unzip '../input/dogs-vs-cats/test1.zip'

In [ ]:
width = 128
height = 128
size = (width, height)
channels = 3
fast_run = False

In [ ]:
filenames  = os.listdir("./train")
catigories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        catigories.append(1)
    else:
        catigories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': catigories
})

In [ ]:
df.head(20)

In [ ]:
len(filenames)

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
sample = random.choice(filenames)
image = load_img('./train/'+sample)
plt.imshow(image)

In [ ]:
model = Sequential([ 
    Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)), 
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
earlystop = EarlyStopping(patience=10)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [ ]:
df['category'] = df['category'].replace({0: 'cat', 1: 'dog'})

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
 validate_df['category'].value_counts()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    './train/',
    x_col='filename',
    y_col='category',
    target_size=size,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
     validate_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
    target_size=size,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
    target_size=size,
    class_mode='categorical'
)

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
# epochs=3 if fast_run else 50
epochs=1
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)


In [ ]:
model.save_weights("model.h5")